# Script: overview data modalities SCAN-B
Author: Lennart Hohmann (translated to Python)
Date: 04.03.2025

In [5]:
# import 
import os
import pandas as pd
import pyreadr

In [ ]:
# set wd
os.chdir(os.path.expanduser("~/PhD_Workspace/Project_PredictRecurrence/"))

In [ ]:
# input paths
infile_1 = ""